In [121]:
from scraping import scraper
import pandas as pd
import numpy as np
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout
from bs4 import BeautifulSoup
import requests
from datetime import datetime, timedelta

In [37]:
train = pd.read_csv('data_1980_2022.csv')

Scraper = scraper(years = [2023])
games = Scraper.fit()
games = games[(games['Date'] >= datetime.today().strftime('%Y-%m-%d')) & (games['Date'] <= datetime.now() + timedelta(days = 7))]
games = games[(games['Result'].isna()) & (games['PTS%'].notna())]
games = games[['Date', 'Home', 'Away'] + Scraper.features]
games

,Date,Home,Away,PTS%,W_1,W_2,W_5,W_10,W_inf,GF_1,...,GA_2,GA_5,GA_10,GA_inf,GD_1,GD_2,GD_5,GD_10,GD_inf,Rest
324,2023-04-29,Crystal Palace,West Ham,0.019571,0.0,-0.5,0.4,-0.2,0.067235,-1.0,...,0.0,-1.0,-0.5,-0.071023,-1.0,-2.5,1.0,0.0,-0.052083,1.0
325,2023-04-29,Brighton,Wolves,0.170707,-2.0,0.0,-0.4,0.1,0.348485,-1.0,...,1.0,1.2,0.1,0.000000,-4.0,-1.0,-0.8,0.4,0.954545,-1.0
326,2023-04-29,Brentford,Nott'ham Forest,0.171717,0.0,0.5,0.2,0.5,0.393939,-1.0,...,-1.5,-0.8,-0.9,-0.515152,0.0,0.5,0.4,1.1,1.151515,0.0
327,2023-04-30,Bournemouth,Leeds United,0.060606,1.0,0.5,0.6,0.2,0.090909,0.0,...,0.5,-1.4,-0.5,0.000000,1.0,-1.0,1.2,0.2,-0.303030,-2.0
328,2023-04-30,Manchester Utd,Aston Villa,0.099707,-1.0,0.0,-0.4,-0.5,0.203324,1.0,...,0.5,0.4,1.0,0.015640,-1.0,0.5,-0.8,-1.4,0.138807,-2.0
329,2023-04-30,Newcastle Utd,Southampton,0.403409,2.0,1.5,1.4,0.9,0.860795,4.0,...,-1.0,-0.8,-0.4,-0.914773,4.0,4.5,3.4,1.9,1.909091,0.0
330,2023-04-30,Fulham,Manchester City,-0.316196,-2.0,-1.0,-1.2,-1.0,-0.612903,-4.0,...,0.0,0.4,0.5,0.408266,-4.0,-2.5,-2.4,-2.2,-1.678427,1.0
331,2023-04-30,Liverpool,Tottenham,0.006629,1.0,1.5,0.8,0.2,0.035985,0.0,...,-2.5,-1.4,-0.7,-0.387311,1.0,3.5,2.4,1.2,0.475379,1.0
332,2023-05-01,Leicester City,Everton,0.010101,1.0,1.0,0.2,-0.2,-0.030303,0.0,...,-1.0,-0.4,-0.4,0.212121,3.0,2.0,0.8,0.4,0.363636,2.0
333,2023-05-02,Arsenal,Chelsea,0.351326,0.0,0.5,0.8,0.9,0.669508,1.0,...,1.5,1.0,0.2,0.057765,-1.0,0.0,1.2,1.8,1.368371,0.0


In [38]:
train = train[(train['Game_home'] > 4) & (train['Game_away'] > 4) & (train['Rest'].abs() < 30)]
X_train, X_pred = np.array(train[Scraper.features]), np.array(games[Scraper.features])
y_train = np.array(train[['Win', 'Draw', 'Loss']])

# Create neural network model
model = Sequential()
model.add(Dense(32, activation = 'relu', input_dim = len(Scraper.features)))
model.add(Dense(16, activation = 'relu'))
model.add(Dropout(0.1))
model.add(Dense(3,  activation = 'softmax'))

# Compile model
model.compile(loss      =  'categorical_crossentropy',
              optimizer =  'adam',
              metrics   = ['accuracy'])

# Train model
history = model.fit(X_train, y_train, epochs = 50, batch_size = 32, validation_split = 0.1)

2023-04-29 17:47:41.001109: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/50
437/437 [==============================] - 1s 852us/step - loss: 1.0493 - accuracy: 0.4641 - val_loss: 1.0060 - val_accuracy: 0.5193
Epoch 2/50
437/437 [==============================] - 0s 573us/step - loss: 1.0220 - accuracy: 0.4933 - val_loss: 1.0122 - val_accuracy: 0.5167
Epoch 3/50
437/437 [==============================] - 0s 533us/step - loss: 1.0170 - accuracy: 0.5011 - val_loss: 1.0009 - val_accuracy: 0.5199
Epoch 4/50
437/437 [==============================] - 0s 601us/step - loss: 1.0157 - accuracy: 0.4986 - val_loss: 1.0215 - val_accuracy: 0.5161
Epoch 5/50
437/437 [==============================] - 0s 522us/step - loss: 1.0142 - accuracy: 0.5010 - val_loss: 1.0055 - val_accuracy: 0.5187
Epoch 6/50
437/437 [==============================] - 0s 523us/step - loss: 1.0126 - accuracy: 0.5009 - val_loss: 1.0099 - val_accuracy: 0.5174
Epoch 7/50
437/437 [==============================] - 0s 523us/step - loss: 1.0126 - accuracy: 0.5008 - val_loss: 1.0125 - val_accuracy:

In [113]:
preds = model.predict(X_pred)
games['pred'] = list(pd.Series(np.argmax(preds, axis = 1)).replace(0, 'HOME').replace(1, 'DRAW').replace(2, 'AWAY'))

1/1 [==============================] - 0s 18ms/step


In [119]:
games[['HOME_W', 'DRAW', 'AWAY_W']] = np.round(preds, 2)

In [120]:
print(games[['Date', 'Home', 'Away', 'pred', 'HOME_W', 'DRAW', 'AWAY_W']])

          Date            Home             Away  pred  HOME_W  DRAW  AWAY_W
324 2023-04-29  Crystal Palace         West Ham  HOME    0.41  0.35    0.24
325 2023-04-29        Brighton           Wolves  HOME    0.53  0.30    0.17
326 2023-04-29       Brentford  Nott'ham Forest  HOME    0.67  0.22    0.11
327 2023-04-30     Bournemouth     Leeds United  HOME    0.66  0.20    0.15
328 2023-04-30  Manchester Utd      Aston Villa  HOME    0.38  0.29    0.33
329 2023-04-30   Newcastle Utd      Southampton  HOME    0.76  0.20    0.04
330 2023-04-30          Fulham  Manchester City  AWAY    0.27  0.22    0.51
331 2023-04-30       Liverpool        Tottenham  HOME    0.69  0.24    0.07
332 2023-05-01  Leicester City          Everton  HOME    0.57  0.24    0.18
333 2023-05-02         Arsenal          Chelsea  HOME    0.58  0.22    0.20


In [125]:
try:
    load_model('NNMODEL.h5')
except:
    print('hey')